In [1]:
!pip install transformers

In [2]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [5]:
# !pip install -q -U bitsandbytes
# !pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, BitsAndBytesConfig, pipeline, set_seed

import torch

In [2]:
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)
    print("CUDA Device Name:", torch.cuda.get_device_name(0))

PyTorch Version: 2.3.0+cu121
CUDA Available: True
CUDA Version: 12.1
CUDA Device Name: Tesla T4


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [35]:
tokenizer = AutoTokenizer.from_pretrained("MahmoudMohamed/Phi3_DPO_MeetingQA_4bit")
model = AutoModelForCausalLM.from_pretrained("MahmoudMohamed/Phi3_DPO_MeetingQA_4bit")

# generator = pipeline('text-generation', model=model, tokenizer=tokenizer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [36]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [37]:
def predict(query):
        # Tokenize the input text with truncation and add padding if necessary
        inputs = tokenizer(
            [
                alpaca_prompt.format(
                    query, # instruction

                    "", # output - leave this blank for generation!
                )
            ], return_tensors="pt").to(device)  # Move inputs to the device

        # Generate text using the model with max_new_tokens to avoid length issues
        outputs = model.generate(
            **inputs,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=512
        )

        # Decode the generated text
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [43]:
prompt = 'How can I improve my public speaking skills?'
print(predict(prompt))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How can I improve my public speaking skills?

### Response:
Improving public speaking skills involves a combination of practice, preparation, and self-awareness. Here are some steps you can take to enhance your public speaking abilities:

1. **Prepare thoroughly:** Research your topic, organize your thoughts, and create a clear outline of your speech.
2. **Practice regularly:** Rehearse your speech multiple times, focusing on clarity, pacing, and delivery.
3. **Seek feedback:** Ask friends, family, or colleagues to provide constructive criticism on your speech and presentation style.
4. **Join a public speaking group:** Organizations like Toastmasters provide a supportive environment for practicing and improving public speaking skills.
5. **Watch and learn:** Observe skilled public speakers, either in person or t